In [45]:
#pip install -r requirements.txt

In [46]:
#pip install deepface

In [47]:
%cd C:\Users\dell\HCI_Project_Phase_2\yolov7

C:\Users\dell\HCI_Project_Phase_2\yolov7


In [48]:
#%cd C:\Users\dell\HCI_Project_Phase_2

In [49]:
#pip install tensorflow==2.10.0

In [50]:
#pip install tensorflow-gpu==2.10.0


In [51]:
#pip uninstall tensorflow

In [52]:
#pip install --upgrade numpy


In [53]:
#pip install tensorflow==2.10.0

Note: you may need to restart the kernel to use updated packages.


In [54]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)
print("TensorFlow-GPU enabled:", tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.10.0
TensorFlow-GPU enabled: True


In [55]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11169218171271786995
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4163895296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6594695589651776879
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [56]:

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # GPU(s) detected
    for gpu in gpus:
        print("GPU:", gpu)
else:
    print("No GPU detected.")


GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [57]:
#%matplotlib inline

In [58]:
#pip install --upgrade deepface

In [66]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from numpy import random
import face_recognition
from gaze_tracking import GazeTracking
import deepface
from deepface import DeepFace

In [67]:
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

In [68]:


def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [69]:
classes_to_filter = ['train'] #You can give list of classes to filter by name ['train','person' ]


opt  = {
    "weights": "weights/yolov7.pt", # Path to weights file default weights are for nano model
    "yaml"   : "data/coco.yaml",
    "img-size": 640, # default image size
    "conf-thres": 0.25, # confidence threshold for inference.
    "iou-thres" : 0.45, # NMS IoU threshold for inference.
    "device" : 'cpu',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    "classes" : classes_to_filter  # list of classes to filter or None
}

In [70]:

# # Video information
# fps = 30  # Adjust the frame rate as needed
# w, h = 640, 480  # Set the desired width and height for the video output

# # Initializing video object
# video = cv2.VideoCapture(0)  # Use index 0 for the default camera

# # Initializing object for writing video output
# #output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (w, h))

# # Initializing model and setting it for inference
# with torch.no_grad():
#     weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
#     imgsz = 416  # Set the input image size for the model

#     set_logging()
#     device = select_device('')
#     half = device.type != 'cpu'
#     model = attempt_load(weights, map_location=device)
#     stride = int(model.stride.max())
#     imgsz = check_img_size(imgsz, s=stride)
#     if half:
#         model.half()

#     names = model.module.names if hasattr(model, 'module') else model.names
#     colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
#     if device.type != 'cpu':
#         model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

#     while True:
#         ret, img0 = video.read()

#         if ret:
#             img = letterbox(img0, imgsz, stride=stride)[0]
#             img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
#             img = np.ascontiguousarray(img)
#             img = torch.from_numpy(img).to(device)
#             img = img.half() if half else img.float()
#             img /= 255.0
#             if img.ndimension() == 3:
#                 img = img.unsqueeze(0)

#             # Inference
#             t1 = time_synchronized()
#             pred = model(img, augment=False)[0]
#             pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.6)

#             t2 = time_synchronized()
#             for i, det in enumerate(pred):
#                 if len(det):
#                     det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

#                     for *xyxy, conf, cls in reversed(det):
#                         label = f'{names[int(cls)]} {conf:.2f}'
#                         plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

#             cv2.imshow('Object Detection', img0)
#             #output.write(cv2.resize(img0, (w, h)))

#             if cv2.waitKey(1) == ord('q'):  # Press 'q' to exit
#                 break
#         else:
#             break

# #output.release()
# video.release()
# cv2.destroyAllWindows()


YOLOR  bf4d028 torch 2.0.1+cpu CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients


In [64]:

# # Initialize video capture object
# gaze = GazeTracking()
# video_capture = cv2.VideoCapture(0)  # Use index 0 for the default camera
# # Load a sample picture and learn how to recognize it.
# Supra_image = face_recognition.load_image_file("Supraa.jpg")
# Supra_face_encoding = face_recognition.face_encodings(Supra_image)[0]
# # Load face recognition model and known face encodings
# known_face_encodings = [Supra_face_encoding]  # Replace with your known face encodings
# known_face_names = ['Supraa']  # Replace with your known face names

# # Load object detection model
# weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
# imgsz = 416  # Set the input image size for the model

# set_logging()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# half = device.type != 'cpu'
# model = attempt_load(weights, map_location=device)
# stride = int(model.stride.max())
# imgsz = check_img_size(imgsz, s=stride)
# if half:
#     model.half()

# names = model.module.names if hasattr(model, 'module') else model.names
# colors = [[np.random.randint(0, 255) for _ in range(3)] for _ in names]
# if device.type != 'cpu':
#     model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

# # Process frames from the video capture
# while True:
#     # Grab a single frame of video
#     ret, frame = video_capture.read()

#     if ret:
#         # Face detection
#         small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
#         rgb_small_frame = small_frame[:, :, ::-1]
#         face_locations = face_recognition.face_locations(rgb_small_frame)
#         face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

#         gaze.refresh(frame)

#         new_frame = gaze.annotated_frame()
#         text = ""
#         if gaze.is_right():
#             text = "Looking right"
#         elif gaze.is_left():
#             text = "Looking left"
#         elif gaze.is_center():
#             text = "Looking center"
#         elif gaze.is_blinking():
#             text = "Blinking"
            
#         cv2.putText(new_frame, text, (60, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)
#         print(text)
#         face_names = []
#         for face_encoding in face_encodings:
#             matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#             name = "Unknown"

#             face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
#             best_match_index = np.argmin(face_distances)
#             if matches[best_match_index]:
#                 name = known_face_names[best_match_index]

#             face_names.append(name)

#         # Object detection
#         img0 = frame
#         img = letterbox(img0, imgsz, stride=stride)[0]
#         img = img[:, :, ::-1].transpose(2, 0, 1)
#         img = np.ascontiguousarray(img)
#         img = torch.from_numpy(img).to(device)
#         img = img.half() if half else img.float()
#         img /= 255.0
#         if img.ndimension() == 3:
#             img = img.unsqueeze(0)

#         # Inference
#         t1 = time_synchronized()
#         pred = model(img, augment=False)[0]
#         pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.6)
#         t2 = time_synchronized()

#         # Display the results
#         for (top, right, bottom, left), name in zip(face_locations, face_names):
#             top *= 4
#             right *= 4
#             bottom *= 4
#             left *= 4

#             cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#             cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#             font = cv2.FONT_HERSHEY_DUPLEX
#             cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

#         for i, det in enumerate(pred):
#             if len(det):
#                 det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

#                 for *xyxy, conf, cls in reversed(det):
#                     label = f'{names[int(cls)]} {conf:.2f}'
#                     plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

#         cv2.imshow('Video', frame)
#         if cv2.waitKey(1) == ord('q'):  # Press 'q' to exit
#             break
#     else:
#         break

# # Release resources
# video_capture.release()
# cv2.destroyAllWindows()


In [65]:

# Initialize video capture object
gaze = GazeTracking()
video_capture = cv2.VideoCapture(0)  # Use index 0 for the default camera
# Load a sample picture and learn how to recognize it.
Supra_image = face_recognition.load_image_file("Supraa.jpg")
Supra_face_encoding = face_recognition.face_encodings(Supra_image)[0]
# Load face recognition model and known face encodings
known_face_encodings = [Supra_face_encoding]  # Replace with your known face encodings
known_face_names = ['Supraa']  # Replace with your known face names

# Load object detection model
weights = 'weights/yolov7.pt'  # Replace with the path to your YOLOv7 weights
imgsz = 416  # Set the input image size for the model

set_logging()
device = select_device('')
half = device.type != 'cpu'
model = attempt_load(weights, map_location=device)
stride = int(model.stride.max())
imgsz = check_img_size(imgsz, s=stride)
if half:
    model.half()

names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

# Process frames from the video capture
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    if ret:
        # Face detection
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        gaze.refresh(frame)
        new_frame = gaze.annotated_frame()
        text = ""
        if gaze.is_right():
            text = "Looking right"
        elif gaze.is_left():
            text = "Looking left"
        elif gaze.is_center():
            text = "Looking center"
        elif gaze.is_blinking():
            text = "Blinking"

        cv2.putText(new_frame, text, (60, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)
        print(text)
        
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

        #Face Expressions
        result = DeepFace.analyze(frame, actions=['emotion'],enforce_detection=False)
        # Retrieve the dominant emotion from the first result in the list
        dominant_emotion = result[0]['dominant_emotion']
        # Display the frame with emotion labels
        cv2.putText(
            frame,
            dominant_emotion,
            (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )
        # Object detection
        img0 = frame
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.6)
        t2 = time_synchronized()

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        for i, det in enumerate(pred):
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

                for *xyxy, conf, cls in reversed(det):
                    label = f'{names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

        cv2.imshow('Video', frame)
        if cv2.waitKey(1) == ord('q'):  # Press 'q' to exit
            break
    else:
        break

# Release resources
video_capture.release()
cv2.destroyAllWindows()


YOLOR  76a11f5 torch 2.0.1+cpu CPU



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 36905341 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 13.05it/s]
